In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#  ALL LIBRARIES USED IN THIS CW
import csv                              
from sklearn.svm import LinearSVC
from nltk.classify import SklearnClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_recall_fscore_support 
import numpy as np
from sklearn.metrics import precision_score
import sklearn
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 
import math

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer()
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from io import TextIOBase
bow=[]
bow_freq=[]
token_freq_in_doc={}
len_slice={}

def load_data(path):
    """Load data from a tab-separated file and append it to raw_data."""
    with open(path,encoding="utf8") as f:
        reader = csv.reader(f, delimiter='\t')
        for line in reader:
            if line[0] == "Id":  
                continue
            (label, text, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts ) = parse_data_line(line)
            
            raw_data.append((text, label, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts))

def split_and_preprocess_data(percentage):
    global bow
    global bow_freq
    global token_freq_in_doc 
    global len_slice

    num_samples = len(raw_data)
    num_training_samples = int((percentage * num_samples))
    t_slice =raw_data[:num_training_samples]

    for text, label, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts in t_slice:
      for word in text.split():
        bow.append(word)

    # remove stopwords in bow and make all words lower case
    stop_words = set(stopwords.words('english'))
    
    bow = [w.lower() for w in bow if not w.lower() in stop_words]
    
    bow_freq = Counter(bow)
    print("Total no. bag of words: ", len(bow))

    # keep only the most common words: threshold set
    new_bow=[]
    for word in bow:
      if bow_freq[word] > 100:
        new_bow.append(word)
        
    print("New total of no. bag of words: ", len(new_bow))
    bow = new_bow
    bow_freq = Counter(bow)
    
    # formula for TF - IDF : TF X IDF
    # calculating IDF
    ct=0
    for token in bow_freq.items():
      ct+=1
      for sen in t_slice:
        # take only  statement and leave out label for statement and only key from token dict
        if token[0] in sen[0].lower():
          # count separate tokens in sentence after split
          t = sen[0].split()
          len_slice[token[0]] = len(sen[0])
          
          if token[0] not in token_freq_in_doc.keys():
              token_freq_in_doc[token[0]] = 0
          token_freq_in_doc[token[0]]+=1
   


    for token in token_freq_in_doc:
      token_freq_in_doc[token] = math.log( (1+ num_training_samples) / (1+ token_freq_in_doc[token]) ) +1

    for (text, label, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts) in raw_data[:num_training_samples]:
        #  print('oof ', total_barely_true_counts[text] )
         train_data.append((to_feature_vector(pre_process(text, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts)
         , total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts ),label))

    for (text, label, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts) in raw_data[num_training_samples:]:
        test_data.append((to_feature_vector(pre_process(text, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts )
         , total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts ),label))

Question 1: Input and Basic preprocessing (10 marks)

In [ ]:
def convert_lbl(label):
    lbl_map = {
        'true': 'REAL',
        'mostly-true': 'REAL',
        'half-true': 'REAL',
        'false': 'FAKE',

        'barely-true': 'FAKE',
        'pants-fire': 'FAKE',

    }
    return lbl_map[label]

# data_line[8][9372] has no items for some lines

def parse_dt_line(data_line):
    label = convert_lbl(data_line[1])
    s = data_line[2]
    total_barely_true_counts = data_line[8]
    total_false_counts = data_line[9]
    total_half_true_counts = data_line[10]
    total_pants_on_fire_counts = data_line[11]
    return (label, s, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts)

In [ ]:
def pre_process(text, total_barely_true_counts, total_false_counts, total_half_true_counts, total_pants_on_fire_counts):
    text = text.lower()
    tokens = text.split()
    # print('rlly ', total_barely_true_counts)
    tokens.append(total_barely_true_counts)
    tokens.append(total_false_counts)
    tokens.append(total_half_true_counts)
    tokens.append(total_pants_on_fire_counts)
    return tokens

Question 2: Basic Feature Extraction (20 marks)

In [ ]:
global_feature_dict=[]


def to_feature_vector(tokens, total_barely_true_count, total_false_counts,  total_half_true_counts, total_pants_on_fire_counts ):
    global bow
    global len_slice
    local_dict={}


    
    tok = tokens[:-4]

    pos_counts = Counter(tok)
    # last token is always total_barely_true_counts and IS NOT part of TF-IDF 
    local_dict['total_barely_true_counts']= total_barely_true_count
    local_dict['total_false_counts']= total_false_counts
    local_dict['total_half_true_counts']= total_half_true_counts
    local_dict['total_pants_on_fire_counts']= total_pants_on_fire_counts
    print(local_dict)
    return {**local_dict,**pos_counts}

Question 3: Cross-validation (20 marks)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def cross_vald(dataset, folds):
    fold_size = int(len(dataset)/folds) +1
    p=[]
    r=[]
    f1=[]
    a=[]
    for i in range(0,len(dataset),int(fold_size)):
        test_set=dataset[i:i+fold_size]
        train_set = dataset[:i]+dataset[i+fold_size:]
       
        classifier = train_classifier(train_set)

        y_test = predict_lbls([x[0] for x in test_set], classifier)
     
        true_label= [x[1] for x in test_set]
        report= classification_report(true_label, y_test, output_dict= True)
        p.append(report['FAKE']['precision'])
        r.append(report['FAKE']['recall'])
        f1.append(report['FAKE']['f1-score'])
        a.append(sklearn.metrics.accuracy_score(true_label, y_test))
     
    return [np.mean(p),np.mean(r), np.mean(f1),np.mean(a)]

In [ ]:
def predict_lbls(samples, classifier):
    return classifier.classify_many(samples)

def predict_lbl_from_raw(sample, classifier):
    return classifier.classify(to_feature_vector(preProcess(reviewSample)))

In [ ]:
# MAIN
nltk.download('omw-1.4')
raw_data = []          
train_data = []        
test_data = []         
data_file_path = 'fake_news.tsv'
load_data(data_file_path) 
split_and_preprocess_data(0.8)


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Streaming output truncated to the last 5000 lines.
{'total_barely_true_counts': '8', 'total_false_counts': '2', 'total_half_true_counts': '10', 'total_pants_on_fire_counts': '8'}
{'total_barely_true_counts': '6', 'total_false_counts': '8', 'total_half_true_counts': '2', 'total_pants_on_fire_counts': '5'}
{'total_barely_true_counts': '10', 'total_false_counts': '17', 'total_half_true_counts': '27', 'total_pants_on_fire_counts': '19'}
{'total_barely_true_counts': '1', 'total_false_counts': '0', 'total_half_true_counts': '0', 'total_pants_on_fire_counts': '0'}
{'total_barely_true_counts': '1', 'total_false_counts': '1', 'total_half_true_counts': '3', 'total_pants_on_fire_counts': '4'}
{'total_barely_true_counts': '5', 'total_false_counts': '5', 'total_half_true_counts': '11', 'total_pants_on_fire_counts': '8'}
{'total_barely_true_counts': '9', 'total_false_counts': '8', 'total_half_true_counts': '10', 'total_pants_on_fire_counts': '18'}
{'total_barely_true_counts': '2', 'total_false_count

In [ ]:
# TRAINING AND VALIDATING OUR CLASSIFIER
def train_classifier(data):
    pipeline =  Pipeline( [ ('svc', LinearSVC( max_iter=1000, C=1) ) ])
    return SklearnClassifier(pipeline).train(data)

In [ ]:
cross_vald(train_data, 10)  # will work and output overall performance of p, r, f-score when cv implemented

[0.6068697544066738,
 0.6075665524374085,
 0.6067435180208932,
 0.6581010452961673]